In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from prefect.blocks.system import Secret
from catnip.fla_redshift import FLA_Redshift
from typing import Dict
from concurrent.futures import ThreadPoolExecutor
import openpyxl 
from catnip.fla_sharepoint import FLA_Sharepoint

In [2]:
nightly = pd.read_excel("C:\\Users\\riffere\\Florida Panthers\\SP-BS - Documents\Data Science\Resources\\2023-24 Temporary Report Data\\23-24 Panthers Nightly Data TSS Temp.xlsx")
plans = pd.read_excel("C:\\Users\\riffere\\Florida Panthers\\SP-BS - Documents\Data Science\Resources\\2023-24 Temporary Report Data\\23-24 Panthers Subscription Data TSS Temp.xlsx")

In [3]:
df = nightly.groupby(by = ['Plan Type'])[['Gross Revenue','Net Revenue','Tickets']].sum()
df.reset_index(inplace = True)
df = df[df['Plan Type'].apply(lambda x: len(str(x))>1)]
df['FSE'] = 0
df1 = nightly.groupby(by = ['Plan Type'])['Customer ID'].nunique()
df1 = df1.to_frame()
df1.reset_index(inplace = True)
df = df.merge(right = df1, how = 'left', on = ['Plan Type'])
df['Gross Paid ATP'] = [x/y for x,y in zip(df['Gross Revenue'],df['Tickets'])]
df['Net Paid ATP'] = [x/y for x,y in zip(df['Net Revenue'],df['Tickets'])]
df['Ticket Type'] = 0
df['Recency'] = 0
df = df[['Plan Type', 'Ticket Type','Recency','Gross Revenue', 'Net Revenue', 'FSE', 'Tickets','Customer ID', 'Gross Paid ATP',
          'Net Paid ATP']].rename(columns = {'Tickets' : 'Paid Seats', 'Customer ID' : 'Unique Accounts'})

In [11]:
df['Gross Revenue'].sum()

2051076.59

In [4]:
df2 = plans.groupby(by = ['Plan Type', 'PC2', 'Ticket Type', 'Recency'])[['Gross Revenue','Net Revenue','Tickets']].sum()
df2.reset_index(inplace = True)
df2 = df2[df2['Plan Type'].apply(lambda x: len(str(x))>1)]
df2['Paid Seats'] = [x*y for x,y in zip([42, 42, 42, 42, 42, 42, 42, 42, 25, 25, 25, 25, 42, 42, 42, 42, 25, 25, 42, 42, 42], df2['Tickets'])]
df2 = df2.groupby(by = ['Plan Type', 'Ticket Type', 'Recency'])[['Gross Revenue','Net Revenue','Tickets', 'Paid Seats']].sum()
df2.reset_index(inplace = True)
df3 = plans.groupby(by = ['Plan Type', 'Ticket Type', 'Recency'])['Customer ID'].nunique()
df3 = df3.to_frame()
df3.reset_index(inplace = True)
df2 = df2.merge(right = df3, how = 'left', on = ['Plan Type', 'Ticket Type', 'Recency'])
df2['Gross Paid ATP'] = [x/y for x,y in zip(df2['Gross Revenue'],df2['Paid Seats'])]
df2['Net Paid ATP'] = [x/y for x,y in zip(df2['Net Revenue'],df2['Paid Seats'])]
df2 = df2.rename(columns = {'Tickets' : 'FSE', 'Customer ID' : 'Unique Accounts'})

In [10]:
df2['Gross Revenue'].sum()

35956592.88999999

In [5]:
logitix = pd.read_excel("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\logitix_2324.xlsx")

In [6]:
dffinal = pd.concat([df,df2,logitix])
dffinal = dffinal[dffinal['Plan Type'].apply(lambda x: len(str(x))>1)]
dffinal = dffinal.reset_index()
goal_list = [1100000,3250000,2250000,4250000,5800000,5800000,22650000,22650000,22650000,22650000,22650000,22650000,
             6600000,6600000,6600000,6600000,1350000,1350000,1350000,1350000,500000,500000,500000,5250000]
dffinal['bugdet_goal'] = goal_list

In [7]:
def get_pi_credentials():

    credentials = {
        "username": "panthersinsights@floridapanthers.com",
        "password": Secret.load("panthersinsights-email-password").get()
    }

    return credentials

def load(pi_creds, df: pd.DataFrame) -> None:

    FLA_Sharepoint(**pi_creds).upload_dataframe(
        df = dffinal,
        folder_path = "Resources/2023-24 Temporary Report Data",
        file_name = "union_2324",
        as_csv = True,
        add_log_date = False
    )

    return None 

with ThreadPoolExecutor(1) as pool:
    pi_creds = pool.submit(lambda: get_pi_credentials()).result()
    load(pi_creds, dffinal)

/sites/SP-BS/Shared Documents/Data Science/Resources/2023-24 Temporary Report Data/union_2324.csv
